Click the button above to load the interactive widgets

# Run logistic model

Fitting a logistic model on total cases of a country. Select the country and check the predicted peak of infections and total population

In [10]:
interact(logistic_covid, 
         covid19_df = fixed(Confirmed),
         country = Dropdown(options=list(Confirmed.columns), value='Italy'));

interactive(children=(Dropdown(description='country', index=84, options=('Afghanistan', 'Albania', 'Algeria', …

Below you can find the code

# Load modules

In [1]:
import pandas as pd
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
# from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit, fsolve

# from IPython.display import Image
from ipywidgets import interact
from ipywidgets import fixed, Dropdown, SelectMultiple

from pandas.tseries.offsets import Week
%matplotlib inline

<b> Preface </b>
    
    Here are being condensed few simple models and data visualization tools that capture my attention in literature. The idea is to play around with it and understand how important is to limit social contact in this period, and also the importance of modelling for understanding and timely actions.

# Load data from JHU

In [2]:
def loadData(fileName):
    data = pd.read_csv(baseURL + fileName) \
             .drop(['Lat', 'Long'], axis=1) #\
#              .melt(id_vars=['Province/State', 'Country/Region'], 
#                  var_name='date', value_name=columnName) \
#              .astype({'date':'datetime64[ns]', columnName:'Int64'}, 
#                  errors='ignore')
    data['Province/State'].fillna('<all>', inplace=True)
#     data[columnName].fillna(0, inplace=True)
    return data

In [3]:
baseURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"

In [4]:
Confirmed = loadData("time_series_covid19_confirmed_global.csv")

# Structure dataset

In [5]:
# grouping by country
Confirmed = Confirmed.groupby(['Country/Region']).sum()

In [6]:
Confirmed = Confirmed.T

In [7]:
Confirmed.index = pd.to_datetime(Confirmed.index)

# Logistic model

Inspired by https://towardsdatascience.com/covid-19-infection-in-italy-mathematical-models-and-predictions-7784b4d7dd8d)

In [8]:
def logistic_model(x,a,b,c):
    return c/(1+np.exp(-(x-b)/a))

In [9]:
def logistic_covid(covid19_df, country):
    """
    df: dataframe of covid19 data with nations as columns and timestamp as index
    country: string
    
    """
    df = covid19_df[country]
    
    x = list(df.reset_index().index)
    y = list(df.values)
    
#     Real data
    plt.scatter(x, y, label="Real data", color="red")
    
#     fit data
    fit = curve_fit(logistic_model, x, y,p0=[50,100,20000])
    errors = [np.sqrt(fit[1][i][i]) for i in [0,1,2]]

    a = fit[0][0]
    b = fit[0][1]
    c = fit[0][2]

#     print('Infection speed '+str(round(a)))
    print('Maximum infections occurred on '+str(pd.to_datetime(df.index[0]) + pd.to_timedelta(round(b), unit='D')))
    print('Total predicted infections '+str(round(c))+ ' +-'+str(round(errors[2])))

    sol = int(fsolve(lambda x : logistic_model(x,a,b,c) - int(c),b))
    pred_x = list(range(max(x),sol))

    # Mdel data
    plt.plot(x+pred_x, [logistic_model(i,fit[0][0],fit[0][1],fit[0][2]) for i in x+pred_x], label="Logistic model");